In [2]:
import os
import csv

In [3]:
# File and Directory Paths
source_dir = "source_files/"
source_metadata_file = source_dir + "papers.csv"
processed_metadata_file = source_dir + "papers_processed.csv"

table_output_dir = "extracted_tables/"
table_code_dir = table_output_dir + "table_code/"
table_result_file = table_output_dir + "tables.csv"

figure_output_dir = "extracted_figures/"
figure_result_file = figure_output_dir + "figures.csv"

In [7]:
# Check paper metadata files for errors
paper_original_row_length = 3
counter_corrupted_rows = 0
with open(source_metadata_file, "r", encoding="utf-8") as paper_metadata:
    spamreader = csv.reader(paper_metadata, delimiter=';', quotechar='"', quoting=csv.QUOTE_ALL)
    for row in spamreader:
        if len(row) != paper_original_row_length:
            counter_corrupted_rows += 1
            print(f"Unexpected row length for: {row}")
print(f"Check for original paper metadata file complete. {counter_corrupted_rows} rows were corrupted.\n\n")
        
paper_processed_row_length = 5
counter_corrupted_rows = 0
with open(processed_metadata_file, "r", encoding="utf-8") as paper_metadata:
    spamreader = csv.reader(paper_metadata, delimiter=";", quotechar='"', quoting=csv.QUOTE_ALL)
    for row in spamreader:
        if len(row) != paper_processed_row_length:
            counter_corrupted_rows += 1
            print(f"Unexpected row length for: {row}")
print(f"Check for processed paper metadata file complete. {counter_corrupted_rows} rows were corrupted.")

Unexpected row length for: ['2409.00764', 'tl', "dr: Chill, y'all: AI Will Not Devour SE", 'cs']
Unexpected row length for: ['2406.01152', 'Learning-based legged locomotion', ' state of the art and future perspectives', 'cs']
Unexpected row length for: ['2406.05140', 'Towards Long Range Detection of Elephants Using Seismic Signals', ' A Geophone-Sensor Interface for Embedded Systems', 'cs']
Unexpected row length for: ['2407.05529', 'Behind the Deepfake: 8% Create', ' 90% Concerned. Surveying public exposure to and perceptions of deepfakes in the UK', 'cs']
Unexpected row length for: ['2407.16601', 'Considering dynamical synergy and integrated information', ' the unusual case of minimum mutual information', 'cs']
Unexpected row length for: ['2311.10487', 'ReuseSense: With Great Reuse Comes Greater Efficiency', ' Effectively Employing Computation Reuse on General-Purpose CPUs', 'cs']
Unexpected row length for: ['2311.11854', 'Asymptotic expansions for the linear PDEs with oscillatory inp

Unexpected row length for: ['2305.07665', 'A Comprehensive Survey on Affective Computing', ' Challenges, Trends, Applications, and Future Directions', 'cs', '0', '0']
Unexpected row length for: ['2305.14364', 'Towards personalised music-therapy', ' a neurocomputational modelling perspective', 'cs', '0', '0']
Unexpected row length for: ['2303.09463', 'An Autonomous System for Head-to-Head Race: Design, Implementation and Analysis', ' Team KAIST at the Indy Autonomous Challenge', 'cs', '0', '0']
Unexpected row length for: ['2408.17436', 'Imprinting New Physics by using Angular profiles of the FCNC process $B_{c}\\to D_{s}^{*}\\left(\\to \\', 'D_{s}\\pi\\right)\\ell^{+}\\ell^{-}$', 'physics', '5', '1']
Unexpected row length for: ['2409.00531', 'Chemical pressure due to impurities in trigonal compounds Eu$T_2Pn_2$ ($T =$ Cd, Zn', ' $Pn =$ P, As, Sb)', 'physics', '5', '7']
Unexpected row length for: ['2409.07095', 'Phase separation bursting and symmetry breaking inside an evaporating drople